The table below shows drought levels in the Lower Mainland basin for 2015 to 2023. Drought levels reached the highest level (level 5) on August 17th this year. In 2022, this occurred at the end of September. In level 5, adverse impacts are almost certain. The lower mainland has experienced significant drought for the past three summers.

In [1]:
#Import libraries
import os
import pandas as pd
import subprocess
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import json
from pandas import json_normalize

In [2]:
os.chdir('C://Users/pmarshal/Documents/Climate-Summary')
os.getcwd()

'C:\\Users\\pmarshal\\Documents\\Climate-Summary'

In [3]:
drought = pd.read_csv('data/drought.csv') 
drought['date'] = pd.to_datetime(dict(year=drought.year, month=drought.month, day=drought.day))

In [4]:
colorscale = alt.Scale(domain=['-0.1', '0', '1', '2', '3', '4', '5'],
                       range=['#1C00ff00', 'lightgreen', 'yellow', 'orange', '#E17F03', 'red', 'darkred'])

alt.Chart(drought).mark_square(size=400).encode(
    alt.X("monthdate(date):T", title=None, axis=alt.Axis(grid=False)),
    alt.Y("year:O", title=None),
    alt.Color("level:Q", title="Drought Level", scale=colorscale, legend=dict(orient="bottom")),
    tooltip=[
        alt.Tooltip("monthdate(date)", title="Date"),
        alt.Tooltip("level", title="Drought Level"),
    ],
).configure_view(
    step=12,
    strokeWidth=0
).configure_axis(
    domain=False
).properties(width=530, height=180)

alt.Chart(...)

In [5]:
# Size of the hexbins
size = 15
# Count of distinct x features
xFeaturesCount = 12
# Count of distinct y features
yFeaturesCount = 7
# Name of the x field
xField = 'date'
# Name of the y field
yField = 'date'

# the shape of a hexagon
hexagon = "M0,-2.3094010768L2,-1.1547005384 2,1.1547005384 0,2.3094010768 -2,1.1547005384 -2,-1.1547005384Z"

chart = alt.Chart(drought).mark_point(size=size**2, shape=hexagon).encode(
    alt.X('monthdate(date):T', title=None, axis=alt.Axis(grid=False, tickOpacity=0, domainOpacity=0)),
    alt.Y('year(date):O', title=None, axis=alt.Axis(labelPadding=20, tickOpacity=0, domainOpacity=0)),
    stroke=alt.value('black'),
    strokeWidth=alt.value(0.2),
    fill=alt.Color('level:Q', scale=colorscale, title='Drought Level')
).transform_calculate(
    # This field is required for the hexagonal X-Offset
    xFeaturePos='(day(datum.' + xField + ') % 2) * ' + str(size * 1.5) + ' + month(datum.' + xField + ') * ' + str(size * 1.5)
).properties(
    # Exact scaling factors to make the hexbins fit
    width=size * xFeaturesCount * 2.31,
    height=size * yFeaturesCount * 2.3094010768,  # 1.7320508076 is approx. sin(60°)*2
).configure_view(
    strokeWidth=0
)
chart

alt.Chart(...)

In [6]:
# Size of the hexbins
size = 15
# Count of distinct x features
xFeaturesCount = 12
# Count of distinct y features
yFeaturesCount = 7
# Name of the x field
xField = 'date'
# Name of the y field
yField = 'date'

# the shape of a hexagon
hexagon = "M0,-2.3094010768L2,-1.1547005384 2,1.1547005384 0,2.3094010768 -2,1.1547005384 -2,-1.1547005384Z"

chart = alt.Chart(drought).mark_point(size=size**2, shape=hexagon).encode(
    alt.X('xFeaturePos:Q', title='Month', axis=alt.Axis(grid=False, tickOpacity=0, domainOpacity=0)),
    alt.Y('year:O', title='Weekday', axis=alt.Axis(labelPadding=20, tickOpacity=0, domainOpacity=0)),
    stroke=alt.value('black'),
    strokeWidth=alt.value(0.2),
    fill=alt.Color('level:Q', title='Drought Level', scale=colorscale),
    tooltip=['month(' + xField + '):O', 'day(' + yField + '):O', 'level:Q']
).transform_calculate(
    # This field is required for the hexagonal X-Offset
    xFeaturePos='(day(datum.' + yField + ') % 2) / 2 + month(datum.' + xField + ')'
).properties(
    # Exact scaling factors to make the hexbins fit
    width=size * xFeaturesCount * 2,
    height=size * yFeaturesCount * 1.7320508076,  # 1.7320508076 is approx. sin(60°)*2
).configure_view(
    strokeWidth=0
)

chart

alt.Chart(...)

### Soil Moisture

The plot below shows the amount of water (cm^3) in a cubic metre of soil, or the soil water content (SWC). Values are derived from satellite data and are averaged over the Capilano Watershed area. Soil moisture levels were at record low levels in late October 2022. This season, soils dried out earlier, but recent rainfall has improved conditions in the last 10 days, 

In [7]:
soil = pd.read_csv('data/wegaw_soil.csv', parse_dates= ['date']) 
soil['year'] = pd.DatetimeIndex(soil['date']).year
soil['month'] = pd.DatetimeIndex(soil['date']).month
soil['DOY'] = pd.DatetimeIndex(soil['date']).dayofyear

In [8]:
title = alt.TitleParams(
   text='Capilano Watershed',
   subtitle="Satellite-derived Soil Moisture",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

soil_line = alt.Chart(soil, title=title).mark_line().encode(
    alt.X('monthdate(date):T', title=None),
    alt.Y('SWC:Q', title='Soil Water Content (cm^3 water/m^3 soil)', scale=alt.Scale(domain=[300000, 450000])),
    alt.Color('year:N', title="Year"),
    tooltip=[alt.Tooltip('date', title="Date"), alt.Tooltip('SWC', title="SWC")]
)

soil_line.properties(width=600)


alt.Chart(...)

In [9]:
df2 = pd.read_csv('data/seyfw_dc.csv', parse_dates= ['datetime']) 
df2['year'] = pd.DatetimeIndex(df2['datetime']).year
df2['month'] = pd.DatetimeIndex(df2['datetime']).month
df2['DOY'] = pd.DatetimeIndex(df2['datetime']).dayofyear

In [10]:
df3 = df2[df2['month'] > 4]
df4 = df3[df3['month'] < 11]

The plot below shows the drought code (DC), which is a component of the fire weather index system and an indicator of seasonal drought effects on forest fuels. This value typical climbs throughout the summer months with the highest values seen late in the summer or early in the fall (driest conditions). This season (bold red line) saw much higher than average values early in the summer, which was similar to 2015 (violet). In 2015, drought codes dropped abruptly at the end of August when a very strong late-summer storm arrived. This year, the sharp drop came in late September. The dashed black line represents average drought codes, which peak at around 275 at the end of August. 

In [11]:
alt.data_transformers.disable_max_rows()

title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Drought Code - 1988-2023",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

highlight = alt.selection(type='single', on='mouseover',
                          fields=['year'], nearest=True, empty="none")

background = alt.Chart(df4[df4['year'] != 2023], title=title).mark_line(opacity=0.3).encode(
    alt.X('monthdate(datetime):T', title=None),
    alt.Y('DC:Q', title='Drought Code'),
    color=alt.condition( highlight, 'year:N', alt.value("lightgray"), legend=None)
).add_selection(
    highlight
)


year_2023 = alt.Chart(df4[df4['year'] == 2023]).mark_line(color= "red", size= 3).encode(
    alt.X('monthdate(datetime):T', title=None),
    alt.Y('DC:Q', title='Drought Code'),
    tooltip=[alt.Tooltip('datetime', title="Date"), alt.Tooltip('DC', title="Drought Code")]
)

year_2015 = alt.Chart(df4[df4['year'] == 2015]).mark_line(color= "violet", opacity=0.7).encode(
    alt.X('monthdate(datetime):T', title=None),
    alt.Y('DC:Q', title='Drought Code'),
    tooltip=[alt.Tooltip('datetime', title="Date"), alt.Tooltip('DC', title="Drought Code")]
)

year_2022 = alt.Chart(df4[df4['year'] == 2022]).mark_line(color= "darkblue", opacity=0.5).encode(
    alt.X('monthdate(datetime):T', title=None),
    alt.Y('DC:Q', title='Drought Code'),
    tooltip=[alt.Tooltip('datetime', title="Date"), alt.Tooltip('DC', title="Drought Code")]
)

mean = alt.Chart(df4).mark_line(color='black', opacity=0.5, strokeDash=[4, 2]).encode(
    alt.X('monthdate(datetime):T', title=None),
    alt.Y('mean(DC)', title='Drought Code')
)

#year_2023 + year_2015 + year_2022 + mean + background.properties(width=600)

C:\Users\pmarshal\Anaconda3\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
C:\Users\pmarshal\Anaconda3\lib\site-packages\altair\vegalite\v5\api.py:425: AltairDeprecationWarning: The types 'single' and 'multi' are now
        combined and should be specified using "selection_point()".
  warnings.warn(
C:\Users\pmarshal\Anaconda3\lib\site-packages\altair\vegalite\v5\api.py:362: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
C:\Users\pmarshal\Anaconda3\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


(label2)=
## Wildfire Conditions

### Provincial Wildfire Summary

Both British Columbia and Canada experienced record-breaking and devastating wildfire seasons. As of October 1, there have been 2220 fires, and 2.85 million hectares of forest has burned this year in BC. This is now double the previous record for area burned, which occurred in 2018. Nationally, over 18 million hectares of forest has burned (compared to a 10-year average of around 2.6 million hectares). Much of the area burned has occurred in the northern boreal forests. 

Below you can see current BC Wildfire Statistics in comparison to past seasons. All fire zones in the province have seen significant fire activity this season. Source: [BCWS](https://www2.gov.bc.ca/gov/content/safety/wildfire-status/about-bcws/wildfire-statistics/wildfire-averages#:~:text=Wildfire%20Averages%20%20%20%20Year%20%20,%20375%20%2845%25%29%20%2012%20more%20rows%20).

In [12]:
fires = pd.read_csv('data/fire_stats.csv', parse_dates= ['date'])
fires['year'] = pd.DatetimeIndex(fires['date']).year

In [13]:
danger = pd.read_csv('data/seyfw_danger_filter.csv', parse_dates= ['datetime']) 
danger['year'] = pd.DatetimeIndex(danger['datetime']).year
danger['month'] = pd.DatetimeIndex(danger['datetime']).month
danger['DOY'] = pd.DatetimeIndex(danger['datetime']).dayofyear
danger['week'] = danger['datetime'].dt.isocalendar().week
danger.head()

,datetime,DGR,rating,year,month,DOY,week
0,2015-01-01,1,Very Low,2015,1,1,1
1,2015-01-02,1,Very Low,2015,1,2,1
2,2015-01-03,1,Very Low,2015,1,3,1
3,2015-01-04,1,Very Low,2015,1,4,1
4,2015-01-05,1,Very Low,2015,1,5,2


In [14]:
danger_filter = danger.loc[(danger['month']>=4)]
danger_filter = danger_filter.loc[(danger_filter['month']<=10)]
danger_filter.head(12)
#danger_filter.to_csv('danger_filter.csv', index=False)

,datetime,DGR,rating,year,month,DOY,week
90,2015-04-01,1,Very Low,2015,4,91,14
91,2015-04-02,1,Very Low,2015,4,92,14
92,2015-04-03,1,Very Low,2015,4,93,14
93,2015-04-04,1,Very Low,2015,4,94,14
94,2015-04-05,1,Very Low,2015,4,95,14
95,2015-04-06,1,Very Low,2015,4,96,15
96,2015-04-07,2,Low,2015,4,97,15
97,2015-04-08,2,Low,2015,4,98,15
98,2015-04-09,2,Low,2015,4,99,15
99,2015-04-10,2,Low,2015,4,100,15


In [15]:
weekly_summary = danger_filter.groupby(['year', 'week']).mean().reset_index()
weekly_summary.tail(10)

C:\Users\pmarshal\AppData\Local\Temp\ipykernel_4448\3054526442.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weekly_summary = danger_filter.groupby(['year', 'week']).mean().reset_index()


,year,week,DGR,month,DOY
305,2024,35,1.857143,8.142857,242.0
306,2024,36,2.857143,9.000000,249.0
307,2024,37,2.428571,9.000000,256.0
308,2024,38,1.714286,9.000000,263.0
309,2024,39,1.000000,9.000000,270.0
310,2024,40,1.000000,9.857143,277.0
311,2024,41,1.142857,10.000000,284.0
312,2024,42,1.000000,10.000000,291.0
313,2024,43,1.000000,10.000000,298.0
314,2024,44,1.000000,10.000000,303.5


In [16]:
week_danger = pd.read_csv('data/danger_weekly.csv') 
week_danger['date'] = pd.to_datetime(dict(year=week_danger.year, month=week_danger.month, day=week_danger.day))
week_danger.tail(17)

,year,month,day,danger,rating,date
235,2023,7,10,5,Extreme,2023-07-10
236,2023,7,17,5,Extreme,2023-07-17
237,2023,7,24,2,Low,2023-07-24
238,2023,7,31,3,Moderate,2023-07-31
239,2023,8,7,3,Moderate,2023-08-07
240,2023,8,14,3,Moderate,2023-08-14
241,2023,8,21,4,High,2023-08-21
242,2023,8,28,4,High,2023-08-28
243,2023,9,4,4,High,2023-09-04
244,2023,9,11,4,High,2023-09-11


In [17]:
colors = alt.Scale(domain=['1', '2', '3', '4', '5'],
                       range=['#527ced', '#91f527', 'yellow', 'darkorange', 'red'])

danger_map = alt.Chart(week_danger).mark_square(size=400).encode(
    alt.X("monthdate(date):O", title=None, axis=alt.Axis(grid=False)),
    alt.Y("year:O", title=None),
    alt.Color("danger:Q", title="Drought Level", scale=colors, legend=None),
    tooltip=[
        alt.Tooltip("monthdate(date)", title="Date"),
        alt.Tooltip("danger", title="Fire Danger"),
    ],
).configure_view(
    step=12,
    strokeWidth=0
).configure_axis(
    domain=False
).properties(width=550, height=180)

danger_map

alt.Chart(...)

In [18]:
just_2024 = danger_filter.loc[(danger_filter['year']==2024)]
just_2024

,datetime,DGR,rating,year,month,DOY,week
3375,2024-04-01,1,Very Low,2024,4,92,14
3376,2024-04-02,1,Very Low,2024,4,93,14
3377,2024-04-03,1,Very Low,2024,4,94,14
3378,2024-04-04,1,Very Low,2024,4,95,14
3379,2024-04-05,1,Very Low,2024,4,96,14
...,...,...,...,...,...,...,...
3584,2024-10-27,1,Very Low,2024,10,301,43
3585,2024-10-28,1,Very Low,2024,10,302,44
3586,2024-10-29,1,Very Low,2024,10,303,44
3587,2024-10-30,1,Very Low,2024,10,304,44


In [19]:
just_2024 = danger_filter.loc[(danger_filter['year']==2024)]
rating_counts_df = just_2024['rating'].value_counts()
#rating_counts_df = pd.DataFrame({'Rating': rating_counts.index, 'Count': rating_counts.values})
# Define a custom order for the ratings
#custom_order = ['Very Low', 'Low', 'Moderate', 'High', 'Extreme']

# Apply the custom order and sort the DataFrame
#rating_counts_df['Rating'] = pd.Categorical(rating_counts_df['rating'], categories=custom_order, ordered=True)
#rating_counts_df = rating_counts_df.sort_values('Rating')

# Define the mapping of 'Rating' to numerical values
#rating_mapping = {'Very Low': 1, 'Low': 2, 'Moderate': 3, 'High': 4, 'Extreme': 5}

# Add the 'danger' column based on the mapping
#rating_counts_df['danger'] = rating_counts_df['Rating'].map(rating_mapping)
#rating_counts_df

In [20]:
colors = alt.Scale(domain=['Very Low', 'Low', 'Moderate', 'High', 'Extreme'],
                       range=['#527ced', '#91f527', 'yellow', 'darkorange', 'red'])
# Create a chart 
danger_bar = alt.Chart(rating_counts_df).mark_bar().encode(
    x=alt.X('Count:Q', title=None),
    y=alt.Y('danger:O', title=None),
    color=alt.Color('Rating', scale=colors)
).properties(width=400, height=80)

# Define a new color scale for the text
text_colors = alt.Scale(domain=['Very Low', 'Low', 'Moderate', 'High', 'Extreme'],
                        range=['black', 'black', 'black', 'black', 'black'])

text = danger_bar.mark_text(
    align='left',
    baseline='middle',
    size=10,
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Count:Q'),  # Format as percentage
    color=alt.condition(
        alt.datum.Count > 0,  # Condition based on the Count value
        alt.value('black'),   # Set text color to black when the condition is true
        alt.value('transparent')  # Set text color to transparent when the condition is false
    )
)
combined = danger_bar + text

#combined

In [21]:
# Define colors for danger_map
colors_map = alt.Scale(domain=['1', '2', '3', '4', '5'],
                      range=['#527ced', '#91f527', 'yellow', 'darkorange', 'red'])

# Create danger_map chart
danger_map = alt.Chart(week_danger).mark_square(size=400).encode(
    alt.X("monthdate(date):O", title=None, axis=alt.Axis(grid=False)),
    alt.Y("year:O", title=None),
    alt.Color("danger:Q", title="Drought Level", scale=colors_map, legend=None),
    tooltip=[
        alt.Tooltip("monthdate(date)", title="Date"),
        alt.Tooltip("danger", title="Fire Danger"),
    ],
).properties(width=550, height=180)

# Define colors for danger_bar
colors_bar = alt.Scale(domain=['Very Low', 'Low', 'Moderate', 'High', 'Extreme'],
                       range=['#527ced', '#91f527', 'yellow', 'darkorange', 'red'])

# Create danger_bar chart
danger_bar = alt.Chart(rating_counts_df).mark_bar().encode(
    x=alt.X('Count:Q', title=None),
    y=alt.Y('Rating:O', title=None),
    color=alt.Color('Rating', scale=colors_bar)
).properties(width=400, height=80)

# Define a new color scale for the text
text_colors = alt.Scale(domain=['Very Low', 'Low', 'Moderate', 'High', 'Extreme'],
                        range=['black', 'black', 'black', 'black', 'black'])

# Add text to the bars with conditional color
#text = danger_bar.mark_text(
#    align='left',
#    baseline='middle',
#    size=10,
 #   dx=3,
 #   color=alt.condition(
#        alt.datum.Count > 0,
#        alt.value('black'),
 #       alt.value('transparent')
 #   )
#).encode(
 #   text=alt.Text('Count:Q')
#)

# Configure danger_bar for compatibility
#danger_bar = danger_bar.configure_view(
#    strokeWidth=0
#).configure_axis(
#    domain=False
#3)

# Combine the charts vertically
#report_plot = alt.vconcat(danger_map, danger_bar + text, spacing=10)
#report_plot

In [22]:
danger_count  = danger_filter.loc[(danger_filter['DGR']>=1)]
df_dgr = danger_count.loc[:,'year':'datetime']
#danger_count = danger_count.groupby(['year', 'DGR']).count().reset_index()
df_dgr = df_dgr.rename(columns={"year":"year", "datetime": "fire_danger"})
#danger_count['moving_avg'] = danger_count['DGR'].rolling(10).mean()
#danger_count

In [23]:
#Area burned chart

bar = alt.Chart(fires).mark_bar(opacity=0.8, color='darkgray', size = 10).encode(
    alt.X('date:T', title=None),
    alt.Y('area_burned:Q', title='Area Burned in Hectares', axis=alt.Axis(format='s')),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('red'),     # highlight a bar with red.
        alt.value('darkgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('area_burned', title="Area Burned")]
).properties(width=350, height=250, title='BC Area Burned per Year')


bar2 = alt.Chart(fires).mark_bar(opacity=0.8, color='red', size = 10).encode(
    alt.X('date:T', title=None),
    alt.Y('fire_no:Q', title='Number of Fires', axis=alt.Axis(format='s')),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('red'),     # highlight a bar with red.
        alt.value('darkgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('fire_no', title="Number of Fires")]
).properties(width=350, height=250, title='BC Number of Fires')


bar | bar2


alt.HConcatChart(...)

### Watershed Wildfire Conditions
Watershed forest fire danger rating is now rated **VERY LOW or LOW** as of September 30, 2023. Recent rainfall has lowered fire danger and significantly reduced the risk of fire starts on the BC South Coast. It's likely the forest fire season is now over for this year. 

The plot below shows the number of days where fire danger was rated moderate (3), high (4) or extreme (5) in the Seymour Watershed. This varies from season to season, but there has been an increasing trend over time, especially for high and extreme ratings. This year saw the most days in high and extreme fire danger, and the second most in moderate or higher (109 total days). 

In [24]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Days rated moderate (3), high (4), and extreme (5) fire danger",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

#domain = [5, 4, 3, 2, 1]  # Reversed domain
#range_ = ['darkred', '#FE3F46', 'orange', 'green', 'blue']  # Reversed range
domain = [1, 2, 3, 4, 5]
range_ = ['blue', 'green', 'orange', '#FE3F46', 'darkred']

test2 = alt.Chart(danger_filter, title=title).mark_bar(opacity=0.7).encode(
    alt.X('year:O', title=None),
    alt.Y('count(DGR):Q', title = 'Total days'),
    alt.Color("DGR:O", title='Danger Rating', scale=alt.Scale(domain=domain, range=range_), sort=alt.SortOrder('descending')),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('count(DGR)', title="Number of Days")]
)

rule2 = alt.Chart(df_dgr).mark_line(color='black', opacity=0.7, strokeDash=[4, 2]).encode(
    alt.X('year:O', title=None),
    alt.Y('moving_avg:Q', title= 'Total days')
)

(test2 + rule2).properties(width=600)

alt.LayerChart(...)

In [25]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Days in High or Extreme Fire Danger",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

bar1 = alt.Chart(df_dgr, title=title).mark_bar().encode(
    alt.X('year:O', title=None),
    alt.Y('fire_danger:Q', title = 'Count of Days'),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('blue'),     # highlight a bar with red.
        alt.value('lightgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('fire_danger', title="Number of Days")]
)

rule1 = alt.Chart(df_dgr).mark_line(color='red', strokeDash=[4, 2]).encode(
    alt.X('year:O', title=None),
    alt.Y('moving_avg:Q', title= 'Count of Days')
)


In [26]:
# Import data for crossover conditions and format dates
df = pd.read_csv('data/seyfw_crossover.csv', parse_dates= ['datetime']) 
df['year'] = pd.DatetimeIndex(df['datetime']).year
df['month'] = pd.DatetimeIndex(df['datetime']).month
df['DOY'] = pd.DatetimeIndex(df['datetime']).dayofyear

In [27]:
# Filter dataframe for crossover days
df_filter = df.loc[(df['temp']>=30) & (df['rh'] <= 30)]

In [28]:
# Count crossover days per year
df_count = df_filter.groupby('year').count().reset_index()

In [29]:
# Filter for one column and change the name of the column
df_co = df_count.loc[:,'year':'datetime']
df_co = df_co.rename(columns={"year":"year", "datetime": "crossover_count"})
df_co['moving_avg'] = df_co['crossover_count'].rolling(10).mean()

In [30]:
# Export dataframe
# df_co.to_csv('seyfw_crossover_counts.csv', index=False)

The plot below shows the number of days where 30/30 crossover conditions were experienced each year. 30/30 crossover refers to days where temperatures exceed 30 degrees Celsius and relative humidity is lower than 30%. These are very hot and dry days! During these conditions it's likely to see explosive fire behaviour. The red dashed line shows the 10-year moving average. In the past twenty years the number of days with crossover conditions has increase from around 4 to 10. This summer we experienced 15 crossover days, which is third behind 2015 and 2018. 

In [31]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Days in Crossover Condition (30/30) per Year",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

bar = alt.Chart(df_co, title=title).mark_bar().encode(
    alt.X('year:O', title=None),
    alt.Y('crossover_count:Q', title = 'Count of Days'),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('blue'),     # highlight a bar with red.
        alt.value('lightgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('crossover_count', title="Number of Days")]
)

rule = alt.Chart(df_co).mark_line(color='red', strokeDash=[4, 2]).encode(
    alt.X('year:O', title=None),
    alt.Y('moving_avg:Q', title= 'Count of Days')
)

(bar + rule).properties(width=600)

alt.LayerChart(...)

### Wildfire Smoke

Smoke from wildfires in western Canada has resulted in very poor air quality in many parts of North America this summer. Earlier in the season smoke from northern BC fires tended to drift east and south. This caused poor air quality in many parts of the prairies and northeastern US states. New York experienced a period of very poor air quality in early June.

Interior BC fires flared up in mid-August. As the weather pattern changed, smoke drifted south and west toward the Lower Mainland. Metro Vancouver issued air quality advisories for August 19-22, and 25-29 due to unhealthy concentration of fine particulate matter. Wildfire smoke continues to plague parts of North America in early October. 



Canadian wildfires have resulted in record carbon emissions in 2023. In fact, total estimated carbon emissions are over three times greater than the previous record, and six times greater than the 20-year average. 

Significant fire activity started in early May and has continued without respite for the last four months. Most provinces in Canada has dealt with challenging wildfire seasons this year. 

Data for the plot below comes from the Copernicus Atmospheric Monitoring Service (CAMS). CAMS uses satellite observations of Fire radiative power (FRP), which is a measure of fire intensity and the heat of active fires, to estimate emissions of the different pollutants that make up the smoke. 

In [32]:
emission = pd.read_csv('data/emission.csv', parse_dates=['date'])


In [33]:
title = alt.TitleParams(
   text='Estimated Fire Carbon Emissions for Canada',
   subtitle="Copernicus Atmospheric Monitoring Service",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

domain = ['2023', 'Mean (2003-2022)']
range_ = ['red', 'gray']

#soil_line.properties(width=600)

emission_line = alt.Chart(emission).mark_line().encode(
    alt.X('monthdate(date):T', title=None),
    alt.Y('carbon:Q', title='Total Carbon Emissions (megatons)', scale=alt.Scale(domain=[0,500])),
    alt.Color('year:N', title=None, scale=alt.Scale(domain=domain, range=range_), legend=alt.Legend(
        orient='none',
        legendX=30, legendY=10,
        direction='horizontal',
        titleAnchor='middle')),
    tooltip=[alt.Tooltip('date', title='Date'), alt.Tooltip('carbon', title="Total Carbon")]
)

emission_line.properties(width=600)

alt.Chart(...)

In [34]:
domain = ['Very Low', 'Low', 'Moderate', 'High', 'Extreme']
range_ = ['blue', 'lightgreen', 'yellow', 'orange', 'red']

test3 = alt.Chart(danger_filter).mark_bar(opacity=0.7).encode(
    alt.X('year:O', title=None),
    alt.Y('count(rating):O', title = 'Total Days (Apr 1 - Oct 31)'),
    alt.Color("rating:O", title='Danger Rating', scale=alt.Scale(domain=domain, range=range_)),
    order=alt.Order('DGR:O', sort='ascending'),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('count(rating)', title="Number of Days")]
).properties(width=500)

text2 = test3.mark_text(
    align='left',
    baseline='middle',
    dx=5,  # Nudges text to right so it doesn't appear on top of the bar
    angle=270
).encode(
    text='precip:Q'
)

test3

alt.Chart(...)